In [103]:
import pandas as pd
import numpy as np
import seaborn as sns

In [104]:
cc_df = pd.read_csv('/aiffel/data/fraud.csv')

In [105]:
pd.set_option('display.max_columns', 50)

In [106]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,Daniel,Escobar,M,61390 Hayes Port,Romulus,MI,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,6f363661ba6b55889e488dd178f2a0af,1325377215,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,Jenna,Brooks,F,50872 Alex Plain Suite 088,Baton Rouge,LA,70808,30.4066,-91.1468,378909,"Designer, furniture",1977-02-22,1654da2abfb9e79a5f99167fc9779558,1325377421,29.737426,-90.853194,0


In [107]:
cc_df.info()  #결측치가 없다.`

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [108]:
cc_df['job'].nunique()  # 직업은 신용거래이상탐지에 중요한 역할을 할 것 같지만, 고유값이 너무많아 분석이 어려울 거 같아 제외.

110

# 각 속성들의 의미
- trans_date_trans_time: 거래가 발생한 날짜와 시간
- cc_num: 신용카드 번호
- merchant: 거래가 발생한 상점 이름
- category: 상점의 카테고리
- amt: 거래 금액
- first: 카드 소유자의 이름 
- last: 카드 소유자의 성
- gender: 카드 소유자의 성별
- street: 카드 소유자의 주소
- city: 카드 소유자의 도시
- state: 카드 소유자의 주
- zip: 카드 소유자의 우편번호
- lat: 카드 소유자의 주소의 위도
- long: 카드 소유자의 주소의 경도
- city_pop: 카드 소유자가 거주하는 도시의 인구 수
- job: 카드 소유자의 직업
- dob: 카드 소유자의 생년월일 -> 나이 계산
- trans_num: 거래 번호
- unix_time: 거래가 발생한 시간을 유닉스 시간으로 표현한 값
- merch_lat: 상점의 위도
- merch_long: 상점의 경도
- is_fraud: 해당 거래가 사기인지 여부를 나타내는 변수 (0: 정상 거래, 1: 사기 거래)

In [109]:
cc_df.describe()  #불필요한 컬럼들을 제거하고 나면 고려해야할만한 것은 amt하나다. 평균이 69인데 최대값이 25086

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.000000,491134.000000,491134.000000,491134.000000,4.911340e+05,4.911340e+05,491134.000000,491134.000000,491134.000000
mean,3.706013e+17,69.050120,50770.532384,37.931230,-90.495619,1.213922e+05,1.358730e+09,37.930272,-90.495411,0.002533
std,1.260229e+18,160.322867,26854.947965,5.341193,12.990732,3.725751e+05,1.819402e+07,5.372986,13.004100,0.050264
min,5.038744e+11,1.000000,1843.000000,24.655700,-122.345600,4.600000e+01,1.325376e+09,23.655789,-123.345106,0.000000
25%,2.131124e+14,8.960000,28405.000000,33.746700,-97.235100,1.228000e+03,1.343087e+09,33.781388,-96.984814,0.000000
50%,3.531130e+15,42.170000,49628.000000,38.507200,-87.591700,5.760000e+03,1.357257e+09,38.545124,-87.573441,0.000000
75%,4.653879e+15,80.330000,75048.000000,41.520500,-80.731000,5.083500e+04,1.374626e+09,41.624294,-80.685567,0.000000
max,4.956829e+18,25086.940000,99323.000000,48.887800,-69.965600,2.906700e+06,1.388534e+09,49.887523,-68.965624,1.000000


# 불필요한 컬럼 제거

In [110]:
cc_df = cc_df.drop(['first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1)

In [111]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0


-  first-last (이름), street(주소), city(도시), state(주) , zip(우편번호), job(직업), trans_num(거래번호), unix_time (거래시간이 이미 있으므로 중복) : 사기 여부를 예측하는 데 직접적인 영향을 미치지 않을 것으로 보이는 컬럼을 제거 (직업 컬럼 고민)

# 새로운 변수 생성 (Feature Engineering)

### StandardScaler
- 평균이 69인데 최대값이 25086이므로 이상탐지를 하는 목적으로 그냥 둘 수도 있으나, 원핫인코딩을 하기 위해 조정
- 스케일링을 통해 변수의 값을 일정한 범위로 조정

In [112]:
from sklearn.preprocessing import StandardScaler

In [113]:
ss = StandardScaler()

##### 1.거래금액 정규화

In [114]:
cc_df['amt_standardized'] = ss.fit_transform(cc_df[['amt']])

In [115]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141


##### 2. 카테고리(상품 )별 정규화

In [116]:
cc_df['cat_amt']= cc_df.groupby('category')['amt'].transform('sum') #카테고리별 판매금액-> 한 상품을 비정상적으로 많이 살경우 이상거래

In [117]:
cc_df.groupby('category')['amt'].transform('sum').max()

5415410.5

In [118]:
cc_df.groupby('category')['amt'].transform('sum').min()  #카테고리별 판매금액도 많이 차이가 나므로, 스케일링 

799354.17

In [119]:
cc_df['cat_amt_standardized'] = ss.fit_transform(cc_df[['cat_amt']])

In [120]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605


### 시간 계산

##### 1. 나이 계산

In [121]:
import datetime

In [122]:
cc_df['dob'] = pd.to_datetime(cc_df['dob'])

In [123]:
this_year = pd.Timestamp.now().year

In [124]:
cc_df['age'] = this_year - cc_df['dob'].dt.year

In [125]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47


##### 2. 거래 시간

In [126]:
#거래 시간
cc_df['trans_date_trans_time'] = pd.to_datetime(cc_df['trans_date_trans_time'])

In [127]:
cc_df['trans_hour'] = cc_df['trans_date_trans_time'].dt.hour #시간으로 계산해놓기

In [128]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0


##### 3. 거래일자

In [129]:
cc_df['trans_day'] = cc_df['trans_date_trans_time'].dt.date

In [130]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01


### 범주화

###### 1. 거래시간

In [131]:
# 거래시간을 범주화 -> 새벽에 신용카드 거래를 한것은 이상거래의 한 징후로 생각할 수 있으므로
# 거래시간을 0-6시 6-12시 12-18시 18-0시로 각각 구분한후 오른쪽 끝을 미포함 시킴

In [132]:
cc_df['time_of_day'] = pd.cut(cc_df['trans_hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

In [133]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day,time_of_day
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01,Night
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01,Night
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01,Night
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01,Night
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01,Night


##### 2. 구매금액

In [134]:
cc_df['amt_category'] = pd.cut(cc_df['amt'], bins=[0, 100, 200, 500, 1000, 2000, np.inf], labels=[1, 2, 3, 4, 5, 6])

# 구매금액도 너무 큰 경우에 이상거래라고 생각할 수 있기 때문에 이경우도 범주화로 나누어줌, np.inf 2000 이상 무한대라는 뜻

In [135]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day,time_of_day,amt_category
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01,Night,2
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01,Night,1
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01,Night,1
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01,Night,1
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01,Night,1


##### 3. 연령대

In [136]:
cc_df['age_category'] = pd.cut(cc_df['age'], bins=[0, 18, 25, 35, 45, 55, 65, np.inf], labels=['0-18', '19-25', '26-35', '36-45', '46-55', '56-65', '65+'])

# 연령대도 고려해서 신용카드 이상거래에 활용

In [137]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day,time_of_day,amt_category,age_category
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01,Night,2,46-55
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01,Night,1,36-45
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01,Night,1,46-55
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01,Night,1,46-55
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01,Night,1,46-55


### 데이터의 위도/경도 정보

In [148]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (킬로미터)
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance


In [149]:
cc_df['distance'] = cc_df.apply(lambda row: haversine(row['lat'], row['long'], row['merch_lat'], row['merch_long']), axis=1)


In [150]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day,time_of_day,amt_category,age_category,distance,customer_trans_count,customer_avg_amt,amt_diff_from_avg_abs,merchant_avg_amt,daily_trans_count,daily_trans_amt
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01,Night,2,46-55,30.212176,4362,56.078113,51.151887,825,10,735.80
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01,Night,1,36-45,84.702120,3657,59.858059,15.148059,815,2,48.78
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01,Night,1,46-55,67.847742,4382,87.328067,41.048067,521,4,137.08
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01,Night,1,46-55,22.303906,3647,64.317839,0.227839,876,5,341.97
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01,Night,1,46-55,79.591943,4372,90.747123,65.167123,652,5,251.44


### 통계적 관점

In [141]:
cc_df['customer_trans_count'] = cc_df.groupby('cc_num')['amt'].transform('count') #고객의 총 거래 횟수

In [142]:
cc_df['customer_avg_amt'] = cc_df.groupby('cc_num')['amt'].transform('mean') # 고객의 평균 거래금액

In [143]:
cc_df['amt_diff_from_avg_abs'] = (cc_df['amt'] - cc_df['customer_avg_amt']).abs() # 평균 거래 금액과의 절대 차이

In [144]:
cc_df['merchant_avg_amt']= cc_df.groupby('merchant')['amt'].transform('count') #상점별 총 거래 횟수-> 특정 상점에서 이상거래가 많이 발생할 수 있음

In [145]:
cc_df['daily_trans_count'] = cc_df.groupby(['cc_num', 'trans_day'])['amt'].transform('count') #일별 거래 횟수

In [146]:
cc_df['daily_trans_amt'] = cc_df.groupby(['cc_num', 'trans_day'])['amt'].transform('sum') #일별 거래 금액

In [147]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_standardized,cat_amt,cat_amt_standardized,age,trans_hour,trans_day,time_of_day,amt_category,age_category,distance,customer_trans_count,customer_avg_amt,amt_diff_from_avg_abs,merchant_avg_amt,daily_trans_count,daily_trans_amt
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,0.238144,5415410.50,2.405053,46,0,2019-01-01,Night,2,46-55,30.212176,4362,56.078113,51.151887,825,10,735.80
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0,-0.151820,5415410.50,2.405053,44,0,2019-01-01,Night,1,36-45,84.702120,3657,59.858059,15.148059,815,2,48.78
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0,-0.142027,1259079.01,-1.191660,50,0,2019-01-01,Night,1,46-55,67.847742,4382,87.328067,41.048067,521,4,137.08
3,2019-01-01 00:20:15,374930071163758,fraud_Deckow-O'Conner,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0,-0.030938,5415410.50,2.405053,53,0,2019-01-01,Night,1,46-55,22.303906,3647,64.317839,0.227839,876,5,341.97
4,2019-01-01 00:23:41,2712209726293386,fraud_Balistreri-Nader,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0,-0.271141,2345398.08,-0.251605,47,0,2019-01-01,Night,1,46-55,79.591943,4372,90.747123,65.167123,652,5,251.44


# 원 핫 인코딩

In [152]:
pd.Series(cc_df.columns)

0     trans_date_trans_time
1                    cc_num
2                  merchant
3                  category
4                       amt
5                    gender
6                       lat
7                      long
8                  city_pop
9                       dob
10                merch_lat
11               merch_long
12                 is_fraud
13         amt_standardized
14                  cat_amt
15     cat_amt_standardized
16                      age
17               trans_hour
18                trans_day
19              time_of_day
20             amt_category
21             age_category
22                 distance
23     customer_trans_count
24         customer_avg_amt
25    amt_diff_from_avg_abs
26         merchant_avg_amt
27        daily_trans_count
28          daily_trans_amt
dtype: object

In [153]:
select_col = [
    'cc_num', 'category', 'amt_standardized', 'gender', 'city_pop', 'age', 'cat_amt_standardized',
    'time_of_day', 'amt_category', 'age_category',
    'distance', 'customer_trans_count', 'customer_avg_amt', 
    'amt_diff_from_avg_abs', 'merchant_avg_amt', 'daily_trans_count', 
    'daily_trans_amt', 'is_fraud'
]


In [154]:
final_df = cc_df[select_col]

In [155]:
final_df = pd.get_dummies(final_df)

In [157]:
final_df.head()

,cc_num,amt_standardized,city_pop,age,cat_amt_standardized,distance,customer_trans_count,customer_avg_amt,amt_diff_from_avg_abs,merchant_avg_amt,daily_trans_count,daily_trans_amt,is_fraud,category_entertainment,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_F,gender_M,time_of_day_Night,time_of_day_Morning,time_of_day_Afternoon,time_of_day_Evening,amt_category_1,amt_category_2,amt_category_3,amt_category_4,amt_category_5,amt_category_6,age_category_0-18,age_category_19-25,age_category_26-35,age_category_36-45,age_category_46-55,age_category_56-65,age_category_65+
0,630423337322,0.238144,149,46,2.405053,30.212176,4362,56.078113,51.151887,825,10,735.80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,4956828990005111019,-0.151820,124967,44,2.405053,84.702120,3657,59.858059,15.148059,815,2,48.78,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,180048185037117,-0.142027,71485,50,-1.191660,67.847742,4382,87.328067,41.048067,521,4,137.08,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
3,374930071163758,-0.030938,31515,53,2.405053,22.303906,3647,64.317839,0.227839,876,5,341.97,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,2712209726293386,-0.271141,378909,47,-0.251605,79.591943,4372,90.747123,65.167123,652,5,251.44,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


# 회고

처음이 데이터를 받고 정보를 확인해봤을때, 컬럼들이 많아서 이게 무슨 의미인지부터 먼저 파악헀다.
평소 신용거래이상탐지에 대해 관심이 있어서 인터넷으로 검색도 많이 해보고, 유튜브도 조금 찾아봤는데
보통은 이것을 실습할떄, 개인정보보호에 의해서 이렇게 상세하게 나오지 않고 unknown으로 컬럼들이 나오고
그것들이 중요도를 스케일링 해서 주어진 채로 머신러닝 모델에 학습시키는 실습이 나왔다.
지금 이 과정은 그러한 데이터를 만들기 위해서 정제하는 과정이라고 생각했고, 어떤 변수들이 이상거래를 설명해줄수 있는 변수인지 생각해봤다.

먼저 직업부분이 고민이 되었다, 직업의 고유값이 100개가 넘어가 이것을 군집화 한다던지 혹은 다른 처리를 해서 설명해줄수 있는
변수로 생각하기는 어렵다고 생각이 들어서 삭제를 헀다.


또 어려웠던 점은 결측치가 없고, 이상치를 판단하는 부분에서 고려해야하는 부분이 amt 즉 amount 금액 하나였는데 이게 이상해야 이상거래가 아닌가 하는 생각이 들어서 처음 사고하는데 조금 어려움이 들었던 것 같다.

그래서 이 거래금액이 가장 중요하니, 모델에 학습시키기 위해서 정규화를 사용해서 정규화를 해줬고
카테고리 별 거래금액의 합도 매우 큰 차이를 보여서 이부분에서 이상거래를 탐지할 수 있을 것 같아서 집계를 한 후 정규화를 해줬다.

다음으로 시간계산을 하여 나이를 계산하여, 젊은 사람이 많은 금액을 결제하는 것은 또 이상거래의 징후라고 보여질 수 있기 때문에 변수로 만들었고, 거래시간과 거래일자를 각각 만들어서 밑에서 통계적인 변수와 범주화 변수를 만드는데 사용할 목적으로 만들었다.

그리고 그것들을 사용해서 범주형 변수와 통계적인 변수를 만들어서 데이터 정제를 마무리 했다.


가장 어려운 점은

쳇 지피티로 -> 위도와 경도로 거리를 구하는 것을 물어봤는데,

from geopy.distance import geodesic

def calculate_distance(row):
    return geodesic((row['lat'], row['long']), (row['merch_lat'], row['merch_long'])).miles

cc_df['distance'] = cc_df.apply(calculate_distance, axis=1)

이렇게 만든다고 해서 적용을 했는데 작동하지 않았다.

!pip install geopy를 해도 이게 공유 클라우드여서 인지 LMS상의 문제인지 모르겠지만 작동하지 않아서 당황했고

서칭으로 직접 함수로 어떻게 구하는지를 찾아서 겨우 작성했다. 

이렇게 작업환경에 따라서 사용할 수 있는 패키지도 달라진다는 것을 깨달았다.


느낀점: 이것저것 많이 찾아보고 3일동안의 실습 내용을 뒤적거리면서 데이터 정제를 어떻게 해야 좋을지, 변수는 무엇으로 해야좋을지
컬럼은 어떻게 없애는게 좋은지 많이 고민해본 하루였다. 하루동안 이것을 만들면서 느낀점은 말그대로 뼈대만 만든 느낌이다.
이러한 변수를 가지고 어떤 변수에 어떻게 중요도를 결정짓고, 어떻게 해서 이상거래를 탐지하냐라고 물어본다면 
자신있게 답변하기 어렵다는 생각을 했고 앞으로도 공부를 많이 해야겠다는 생각이 들었다. 

마지막 프로젝트에서 rating으로 이러한 변수들의 중요도를 파악하고 또 가중치하는 작업을 배웠었는데, 지금 내가 제출한 프로젝트에 
변수들의 중요도를 결정하고 그것을 가중치로 변환하여 진짜로 머신러닝에 돌릴 수 있는 데이터 프레임을 만드는게 실전에서 아마 해야할 일이라고 생각한다. 

하지만 평소 관심있던 분야가 금융 게임인데 여기서 사용된다는 이상 징후를 어떻게 하는지 맛보기로 배운거 같아서 좋았고
더 열심히 공부해서 데이터 정제가 익숙해져야겠다는 생각을 했다. 